In [1]:
import numpy as np
import random

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models

In [2]:
random.seed(1)
np.random.seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
data_dir = '/content/drive/MyDrive/Untitled folder/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

## 1st Approach - No Pretrained Model

In [6]:
from torchvision.transforms.transforms import CenterCrop
trans = transforms.Compose([transforms.Resize(256),
                            transforms.CenterCrop(256),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406],
                                                 [0.229, 0.224, 0.225])])
training_set = datasets.ImageFolder(train_dir, transform = trans)
validation_set = datasets.ImageFolder(valid_dir, transform = trans)
testing_set = datasets.ImageFolder(test_dir, transform = trans)

train_loader = torch.utils.data.DataLoader(training_set, batch_size = 64, shuffle = True)
validate_loader = torch.utils.data.DataLoader(validation_set, batch_size = 32)
test_loader = torch.utils.data.DataLoader(testing_set, batch_size = 32)

In [7]:
model = models.vgg16(pretrained = False)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
classifier = nn.Sequential(nn.Linear(25088, 4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.5, inplace=False),
                           nn.Linear(4096, 4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.5, inplace=False),
                           nn.Linear(4096, 102),
                           nn.LogSoftmax(dim=1))
model.classifier = classifier
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
def validation(model, validate_loader, criterion):
  val_loss = 0
  accuracy = 0
  for images, labels in iter(validate_loader):
    images, labels = images.to('cuda'), labels.to('cuda')
    output = model.forward(images)
    val_loss += criterion(output, labels).item()
    probabilities = torch.exp(output)
    equality = (labels.data == probabilities.max(dim=1)[1])
    accuracy += equality.type(torch.FloatTensor).mean()
  return val_loss, accuracy

In [15]:
import time

In [24]:
def train_model():
  max_epoch = 10
  print_every = 20
  n_steps = 0
  model.to('cuda')

  start_train = time.time()
  for n_epochs in range(max_epoch):
    print(f'- Epoch {n_epochs + 1}')
    start_epoch = time.time()
    model.train()

    epoch_loss = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
      images, labels = images.to('cuda'), labels.to('cuda')

      optimizer.zero_grad()

      output = model.forward(images)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()
      if n_steps % print_every == 0:
        model.eval()
        with torch.no_grad():
          validation_loss, accuracy = validation(model, validate_loader, criterion)

        print(f"Batch: {batch_idx}/{len(train_loader)}..",
              f"Training Loss: {epoch_loss/print_every:.3f}..",
              f"Validation Loss: {validation_loss/len(validate_loader):.3f}..",
              f"Validation Accuracy: {accuracy/len(validate_loader):.3f}")
         
        epoch_loss = 0
        model.train()
      n_steps += 1
    end_epoch = time.time()
    print(f'--> Epoch {n_epochs + 1} costs {end_epoch - start_epoch} seconds')
  end_train = time.time()
  print(f'Training time: {end_train - start_train}')   

In [25]:
train_model()

- Epoch 1
Batch: 0/103.. Training Loss: 25.030.. Validation Loss: 4.614.. Validation Accuracy: 0.034
Batch: 20/103.. Training Loss: 4.736.. Validation Loss: 4.555.. Validation Accuracy: 0.025
Batch: 40/103.. Training Loss: 4.562.. Validation Loss: 4.558.. Validation Accuracy: 0.034
Batch: 60/103.. Training Loss: 4.535.. Validation Loss: 4.518.. Validation Accuracy: 0.025
Batch: 80/103.. Training Loss: 4.530.. Validation Loss: 4.512.. Validation Accuracy: 0.025
Batch: 100/103.. Training Loss: 4.543.. Validation Loss: 4.508.. Validation Accuracy: 0.025
--> Epoch 1 costs 1916.2193830013275 seconds
- Epoch 2
Batch: 17/103.. Training Loss: 4.050.. Validation Loss: 4.514.. Validation Accuracy: 0.025
Batch: 37/103.. Training Loss: 4.532.. Validation Loss: 4.511.. Validation Accuracy: 0.025
Batch: 57/103.. Training Loss: 4.523.. Validation Loss: 4.505.. Validation Accuracy: 0.034
Batch: 77/103.. Training Loss: 4.525.. Validation Loss: 4.505.. Validation Accuracy: 0.034
Batch: 97/103.. Training

In [28]:
def save_checkpoint(model):
  model.class_to_idx = training_set.class_to_idx
  checkpoint = {'arch': "vgg16",
                'class_to_idx': model.class_to_idx,
                'model_state_dict': model.state_dict()}
  torch.save(checkpoint, '/content/drive/MyDrive/Untitled folder/PyTorch/checkpoint.pth')


In [29]:
save_checkpoint(model)

In [30]:
def test_model(model, test_loader):
  model.eval()
  model.to('cuda')
  with torch.no_grad():
    accuracy = 0
    for images, labels in iter(test_loader):
      images, labels = images.to('cuda'), labels.to('cuda')
      output = model.forward(images)
      probabilities = torch.exp(output)
      equality = (labels.data == probabilities.max(dim=1)[1])
      accuracy += equality.type(torch.FloatTensor).mean()
    print("Test Accuracy: {}".format(accuracy/len(test_loader)))

In [31]:
test_model(model, test_loader)

Test Accuracy: 0.030048076063394547


## 2nd Approach - Pretrained Model

In [34]:
model = models.vgg16(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [35]:
for parameter in model.parameters():
  parameter.requires_grad = False

classifier = nn.Sequential(nn.Linear(25088,4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.5, inplace=False),
                           nn.Linear(4096,4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.5, inplace=False),
                           nn.Linear(4096,102),
                           nn.LogSoftmax(dim=1)
                           )
model.classifier = classifier
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [36]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [37]:
def validation(model, validateloader, criterion):
  val_loss = 0
  accuracy = 0
  for images, labels in iter(validateloader):
    images, labels = images.to('cuda'), labels.to('cuda')
    output = model.forward(images)
    val_loss += criterion(output, labels).item()
    probabilities = torch.exp(output)
    equality = (labels.data == probabilities.max(dim=1)[1])
    accuracy += equality.type(torch.FloatTensor).mean()
  return val_loss, accuracy

In [38]:
import time
def train_model():
  max_epoch = 10
  print_every = 20
  n_steps = 0
  model.to('cuda')

  start_train = time.time()
  for n_epochs in range(max_epoch):
    print(f'- Epoch {n_epochs + 1}')
    start_epoch = time.time()
    model.train()

    epoch_loss = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
      images, labels = images.to('cuda'), labels.to('cuda')
      optimizer.zero_grad()

      output = model.forward(images)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      
      epoch_loss += loss.item()
      if n_steps % print_every == 0:
        model.eval()
        #Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
          validation_loss, accuracy = validation(model, validate_loader, criterion)

        print(f"Batch: {batch_idx}/{len(train_loader)}.. ",
              f"Training Loss: {epoch_loss/print_every:.3f}.. ",
              f"Validation Loss: {validation_loss/len(validate_loader):.3f}.. ",
              f"Validation Accuracy: {accuracy/len(validate_loader):.3f}")
        
        epoch_loss = 0
        model.train()
      n_steps += 1
    end_epoch = time.time()
    print(f'--> Epoch {n_epochs + 1} costs {end_epoch - start_epoch} seconds')
  end_train = time.time()
  print(f"Training time: {end_train - start_train}")

In [39]:
train_model()

- Epoch 1
Batch: 0/103..  Training Loss: 0.233..  Validation Loss: 9.538..  Validation Accuracy: 0.025
Batch: 20/103..  Training Loss: 5.368..  Validation Loss: 3.426..  Validation Accuracy: 0.282
Batch: 40/103..  Training Loss: 2.996..  Validation Loss: 2.016..  Validation Accuracy: 0.533
Batch: 60/103..  Training Loss: 2.174..  Validation Loss: 1.413..  Validation Accuracy: 0.635
Batch: 80/103..  Training Loss: 1.853..  Validation Loss: 1.271..  Validation Accuracy: 0.669
Batch: 100/103..  Training Loss: 1.708..  Validation Loss: 0.979..  Validation Accuracy: 0.768
--> Epoch 1 costs 520.4334514141083 seconds
- Epoch 2
Batch: 17/103..  Training Loss: 1.197..  Validation Loss: 0.882..  Validation Accuracy: 0.770
Batch: 37/103..  Training Loss: 1.335..  Validation Loss: 0.894..  Validation Accuracy: 0.761
Batch: 57/103..  Training Loss: 1.230..  Validation Loss: 0.805..  Validation Accuracy: 0.791
Batch: 77/103..  Training Loss: 1.235..  Validation Loss: 0.714..  Validation Accuracy: 0.

In [40]:
def save_checkpoint(model):
  model.class_to_idx = training_set.class_to_idx
  checkpoint = {'arch': "vgg16",
                'class_to_idx': model.class_to_idx,
                'model_state_dict': model.state_dict()
                }
  torch.save(checkpoint,'/content/drive/MyDrive/Untitled folder/PyTorch/checkpoint.pth')

save_checkpoint(model)

In [41]:
def test_model(model, test_loader):
  model.eval()
  model.to('cuda')

  with torch.no_grad():
    accuracy = 0
    for images, labels in iter(test_loader):
      images, labels = images.to('cuda'), labels.to('cuda')
      output = model.forward(images)
      probabilities = torch.exp(output)
      equality = (labels.data == probabilities.max(dim=1)[1])
      accuracy += equality.type(torch.FloatTensor).mean()
    print("Test Accuracy: {}".format(accuracy/len(test_loader)))

In [42]:
test_model(model, test_loader)

Test Accuracy: 0.8199645280838013
